In [1]:
import sys
sys.path.append("../../")

import os
from dotenv import load_dotenv
import json

from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from self_query_summarization.query_constructor.prompt import *

In [2]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
ATTRIBUTES = """ 
{
    "attributes": {
        "year": {
            "type": "integer",
            "description": "The year the movie was released"
        },
        "title": {
            "type": "string",
            "description": "The title of the movie"
        },
        "genre": {
            "type": "string",
            "description": "The genre of the movie in lowercase. It can contains multiple genres""
        },
        "director": {
            "type": "string",
            "description": "The name of the movie director. It can be multiple names""
        },
        "cast": {
            "type": "string",
            "description": "The name of the actors in the movie""
        },
        "country": {
            "type": "string",
            "description": "The country where the movie was produced""
        },
    }
}
"""

USER_QUERY = "Horror movie from the 90s about young people going to vacation on a barren island. One of them, a female artist, has dreams that depict ghastly murders"

In [28]:
llm = OpenAI()

final_prompt = get_prompt(ATTRIBUTES, USER_QUERY)


results = llm.predict(final_prompt)

print(results)

{
    "query": "Horror movie from the 90s about young people going to vacation on a barren island having nightmares about murders",
    "filter": {
        "$and": [
            {"year": {"$gt": 1990} },
            {"year": {"$lt": 2000} },
            {"genre": {"$in": ["horror"]} }
        ]
    }
}



In [31]:
results = '{\n    "query": "Horror movie from the 90s about young people going to vacation on a barren island having nightmares about murders",\n    "filter": "test" }'

In [32]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

# Define your desired data structure.
class Query(BaseModel):
    query: str = Field(description="text string to compare to document contents")
    filter: dict = Field(description="logical condition statements for filtering documents and metadata")
    
    
parser = PydanticOutputParser(pydantic_object=Query)

parser.invoke(results)

Query(query='Horror movie from the 90s about young people going to vacation on a barren island having nightmares about murders', filter='test')